<a href="https://colab.research.google.com/github/miguelgutierrez/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Miguel_Gutierrez_AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AG2. Algoritmos de optimización Miguel Gutiérrez: https://colab.research.google.com/drive/1mUy7iu2m2sddbG56uyIgTilh6JjsMNJX

In [0]:
TARIFAS = [
[0,5,4,3,999,999,999],
[999,0,999,2,3,999,111],
[999,999,0,1,999,4,10],
[999,999,999,0,5,6,9],
[999,999,999,999,0,999,4],
[999,999,999,999,999,0,3],
[999,999,999,999,999,999,0]
]

def Precios(TARIFAS):
  #TOTAL DE nodos
  N = len(TARIFAS[0])

  #Inicialización de la tabla de precios
  PRECIOS = [ [9999]*N for i in [9999]*N] #n x n
  RUTA = [ [""]*N for i in [""]*N]
  
  for i in range(N-1):
    for j in range(i+1, N):
      MIN = TARIFAS[i][j]
      RUTA[i][j] = i
      
      for k in range(i, j):
        if PRECIOS[i][k] + TARIFAS[k][j] < MIN:
            MIN = min(MIN, PRECIOS[i][k] + TARIFAS[k][j] )
            RUTA[i][j] = k
        PRECIOS[i][j] = MIN
        
  return PRECIOS,RUTA


In [15]:
PRECIOS,RUTA = Precios(TARIFAS)  
print(PRECIOS[0][6])

print("PRECIOS")
for i in range(len(TARIFAS)):
  print(PRECIOS[i])

print("\nRUTA")  
for i in range(len(TARIFAS)):
  print(RUTA[i]) 

def calcular_ruta(RUTA, desde, hasta):
  if desde == hasta:
    #print("Ir a :" + str(desde))
    return desde 
  else:
    return str(calcular_ruta(RUTA, desde, RUTA[desde][hasta])) +  ',' + str(RUTA[desde][hasta]) 

print("\nLa ruta es:")  
calcular_ruta(RUTA, 0,6) 

11
PRECIOS
[9999, 5, 4, 3, 8, 8, 11]
[9999, 9999, 999, 2, 3, 8, 7]
[9999, 9999, 9999, 1, 6, 4, 7]
[9999, 9999, 9999, 9999, 5, 6, 9]
[9999, 9999, 9999, 9999, 9999, 999, 4]
[9999, 9999, 9999, 9999, 9999, 9999, 3]
[9999, 9999, 9999, 9999, 9999, 9999, 9999]

RUTA
['', 0, 0, 0, 1, 2, 5]
['', '', 1, 1, 1, 3, 4]
['', '', '', 2, 3, 2, 5]
['', '', '', '', 3, 3, 3]
['', '', '', '', '', 4, 4]
['', '', '', '', '', '', 5]
['', '', '', '', '', '', '']

La ruta es:


'0,0,2,5'

**Problema de Asignación de tareas**

In [0]:
#Problema de Asignación de tareas
#COSTES=[[11,12,18,40],
#        [14,15,13,22],
#        [11,17,19,23],
#        [17,14,20,28]]

In [0]:
import random

dimension = 16
COSTES_VACIO=[[0 for x in range(dimension)] for y in range(dimension)]

def matrizAleatoria(A):
    for i in range(len(A)):
        for j in range(len(A[0])):
            A[i][j] = random.uniform(0, 1000)
    return A

COSTES = matrizAleatoria(COSTES_VACIO)
#print("COSTES: ", COSTES)

In [89]:
def valor(S,COSTES):
  VALOR = 0
  for i in range(len(S)):
    VALOR += COSTES[S[i]][i] 
  return VALOR
  
valor((0, 1, 2, 3),COSTES)


2184.590477942345

**Fuerza Bruta**

In [0]:
import itertools
from itertools import permutations

def fuerza_bruta(COSTES):
  #Rrepresentación de la solución será una tubpla donde cada valor en la coordernada i-esima es la tarea asignada al agente i
  #Ejemplo (1,2,3,4) Tiene valor 11+15+19+28=73
  #¿Cuántas posibilidades hay? n! -> Complejidad factorial (exponencial)
  #Con dimensión 10: 4,8 sgs.
  #Con dimensión 11: 58,3 sgs.
  #Con dimensión 12: DEMASIADO

  mejor_valor = 10e10
  mejor_solucion = ()
  for s in list(itertools.permutations(range(len(COSTES)))):
    #print(s,valor(s,COSTES))
    valor_tmp = valor(s,COSTES)
    if valor_tmp < mejor_valor:
      mejor_valor = valor_tmp
      mejor_solucion = s
  print("La mejor solución es: ", mejor_solucion, " con valor:", mejor_valor)

In [104]:
import time
start_time = time.time()
fuerza_bruta(COSTES)
print("Duración Fuerza bruta con dimensión ", dimension, " --- %s seconds ---" % (time.time() - start_time))

La mejor solución es:  (7, 3, 2, 6, 4, 1, 8, 0, 9, 5)  con valor: 1647.9802223259505
Duración Fuerza bruta con dimensión  10  --- 5.118406772613525 seconds ---


**Ramificación y poda**


In [0]:
#Coste inferior para soluciones parciales
#  (1,3,) Se asigna la tarea 1 al agente 0 y la tarea 3 al agente 1

def CI(S,COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]

  #Estimacion
  for i in range(len(COSTES)):
    if i not in S:
      VALOR += min( [ COSTES[j][i] for j in range(len(S), len(COSTES))  ])
  return VALOR

def CS(S,COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]

  #Estimacion
  for i in range(len(COSTES)):
    if i not in S:
      VALOR += max( [ COSTES[j][i] for j in range(len(S), len(COSTES))  ])
  return VALOR

#Genera tantos hijos como posibilidades haya para la siguiente elemento de la tupla
#(0,) -> (0,1), (0,2), (0,3)
def crear_hijos(NODO, N):
  HIJOS = []
  for i in range(N):
    if i not in NODO:
      HIJOS.append({'s':NODO +(i,)    })
  return HIJOS
  

In [0]:
def ramificacion_y_poda(COSTES):
  #Con dimensión 10: 0,1 sgs.
  #Con dimensión 11: 0,3 sgs.
  #Con dimensión 15: 1 sgs, 4 sgs, 5 sgs
  #Con dimensión 16: 1 sgs,  12 sgs. algunas veces demasiado
  #Con dimensión 17: DEMASIADO

#Construccion iterativa de soluciones(arbol). En cada etapa asignamos un agente(ramas).
#Nodos del grafo  { s:(1,2),CI:3,CS:5  }
  #print(COSTES)
  DIMENSION = len(COSTES)  
  MEJOR_SOLUCION=tuple( i for i in range(len(COSTES)) )
  CotaSup = valor(MEJOR_SOLUCION,COSTES)
  #print("Cota Superior:", CotaSup)

  NODOS=[]
  NODOS.append({'s':(), 'ci':CI((),COSTES)    } )

  iteracion = 0

  while( len(NODOS) > 0):
    iteracion +=1

    nodo_prometedor = [ min(NODOS, key=lambda x:x['ci']) ][0]['s']
    #print("Nodo prometedor:", nodo_prometedor)

    #Ramificacion
    #Se generan los hijos
    HIJOS =[ {'s':x['s'], 'ci':CI(x['s'], COSTES)   } for x in crear_hijos(nodo_prometedor, DIMENSION) ]

    #Revisamos la cota superior y nos quedamos con la mejor solucion si llegamos a una solucion final
    NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION  ]
    if len(NODO_FINAL ) >0: 
      #print("\n********Soluciones:",  [x for x in HIJOS if len(x['s']) == DIMENSION  ] )
      if NODO_FINAL[0]['ci'] < CotaSup:
        CotaSup = NODO_FINAL[0]['ci']
        MEJOR_SOLUCION = NODO_FINAL
 
    #Poda
    HIJOS = [x for x in HIJOS if x['ci'] < CotaSup   ]

    #Añadimos los hijos 
    NODOS.extend(HIJOS) 

    #Eliminamos el nodo ramificado
    NODOS =  [  x for x in NODOS if x['s'] != nodo_prometedor    ]
   
  print("La solucion final es:" ,MEJOR_SOLUCION , " en " , iteracion , " iteraciones" , " para dimension: " ,DIMENSION  )
  

In [156]:
start_time = time.time()
ramificacion_y_poda(COSTES)
print("Duración Ramificación y Poda con dimension ", dimension, " --- %s seconds ---" % (time.time() - start_time))

KeyboardInterrupt: ignored